In [10]:
# Local application imports
from utils import postgresListDatabases, postgresListTables, postgresImportFile, postgresRunSqlScript

In [15]:
db_name='weather_wizards'

sql_file = '/Users/jhammans/Documents/RutgersBootcamp/GitHub/weather_wizards/sql/tornados_and_radars.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

table_name = 'swdiws_nx3tvs_geojson'
file_location = '/Users/jhammans/Documents/RutgersBootcamp/GitHub/DataStormers/data/swdiws_nx3tvs_geojson.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")     

table_name = 'radar_locations'
file_location = '/Users/jhammans/Documents/RutgersBootcamp/GitHub/DataStormers/data/radar_locations.csv'
load_count = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"Loaded {load_count} records to {table_name}")

databases = postgresListDatabases()
for database in databases:
    print(f"{database}")
    tables = postgresListTables(database)
    for table in tables:
        print(f"\t{table}")

Successfully executed 4 commands from /Users/jhammans/Documents/RutgersBootcamp/GitHub/weather_wizards/sql/tornados_and_radars.sql.
Loaded 2500 records to swdiws_nx3tvs_geojson
Loaded 192 records to radar_locations
animals_db
	cities
	people
city_info
	actor
	address
	articles
	authors
	city
	country
	customer
	customer_list
	film
	film_actor
	inventory
	payment
	rental
	staff
	store
ph_employees
	departments
	dept_emp
	dept_managers
	employees
	salaries
	titles
pets_db
	owner
	pet
	service
surfers
	measurement
	station
	surfboards
	surfers
crowdfunding_db
weather_wizards
	radar_locations
	swdiws_nx3tvs_geojson
